# Make bound and free states from the prepped inputs

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties
running on node: dig139


### Set working directory to the root of the crispy_shifty repo
TODO set to projects dir

In [2]:
os.chdir("/home/pleung/projects/crispy_shifty")
# os.chdir("/projects/crispy_shifty")

### TODO
TODO

In [3]:
# sys.path.insert(0, os.getcwd())
# from crispy_shifty.utils.io import parse_scorefile_linear


# def create_tasks(scaffolds, options):
#     for scaffold in scaffolds.index:
#         # determine where to split the scaffold by counting the number of helixes
#         num_helices = len(scaffolds.loc[scaffold]["topo"])
#         # split even numbers in half
#         if num_helices % 2 == 0:
#             pre_break_helices = [int(num_helices / 2)]
#         # get middle two for odd numbers
#         else:
#             first_helix = int(num_helices / 2)  # rounds down
#             pre_break_helices = [first_helix, first_helix + 1]
#         for pre_break_helix in pre_break_helices:
#             tasks = {}
#             tasks["clash_cutoff"] = 5000
#             tasks["extra_options"] = options
#             # interfaces must be a ratio of 1:3 or 3:1 between the n and c term halves and the bound helix
#             tasks["int_cutoff"] = 0.33
#             # get the name of the original design
#             tasks["name"] = (
#                 scaffolds.loc[scaffold]["pdb"].split("/")[-1].replace(".pdb", "", 1)
#             )
#             tasks["pdb_path"] = scaffold
#             tasks["pre_break_helix"] = pre_break_helix
#             yield tasks


# scaffolds = parse_scorefile_linear(
#     os.path.join(os.getcwd(), "scaffolds/01_prep_inputs/scores.json")
# )

### TODO

In [4]:
# # Python standard library
# import os
# import pwd
# import socket
# import sys

# # 3rd party library imports
# from dask.distributed import Client
# from dask_jobqueue import SLURMCluster

# # Rosetta library imports
# from pyrosetta.distributed.cluster.core import PyRosettaCluster

# # Custom library imports
# sys.path.insert(0, os.getcwd())
# from crispy_shifty.protocols.states import (
#     make_bound_states,
# )  # the functions we will distribute


# print(
#     "run the following from your local terminal to port forward the dashboard to localhost"
# )
# print(
#     f"ssh -L 8000:localhost:8787 {pwd.getpwuid(os.getuid()).pw_name}@{socket.gethostname()}"
# )
# print("dashboard is now visible at localhost:8000")
# print(f"can also view dashboard at {socket.gethostname()}:8787 without port forwarding")
# options = {
#     "-out:level": "200",  # warning outputs only
# }
# output_path = os.path.join(os.getcwd(), "scaffolds/02_make_bound_states")
# os.makedirs(output_path, exist_ok=True)

# if __name__ == "__main__":
#     # configure SLURM cluster as a context manager
#     with SLURMCluster(
#         cores=1,
#         processes=1,
#         job_cpu=1,
#         memory="8GB",
#         queue="short",
#         walltime="3:00:00",
#         death_timeout=120,
#         local_directory="$TMPDIR",  # spill worker litter on local node temp storage
#         log_directory="/mnt/home/pleung/logs/slurm_logs",
#         extra=["--lifetime", "3h", "--lifetime-stagger", "5m"],
#     ) as cluster:
#         print(cluster.job_script())
#         # scale between 1-150 workers,
#         cluster.adapt(
#             minimum=1,
#             maximum=150,
#             wait_count=999,  # Number of consecutive times that a worker should be suggested for removal it is removed
#             interval="5s",  # Time between checks
#             target_duration="60s",
#         )
#         # setup a client to interact with the cluster as a context manager
#         with Client(cluster) as client:
#             print(client)
#             client.upload_file(
#                 os.path.join(os.getcwd(), "crispy_shifty/protocols/states.py")
#             )  # upload the script that contains the functions to distribute
#             PyRosettaCluster(
#                 client=client,
#                 logging_level="WARNING",
#                 output_path=output_path,
#                 project_name="crispy_shifty",
#                 scratch_dir=output_path,
#                 simulation_name="notebooks_02_make_bound_states",
#                 tasks=create_tasks(scaffolds, options),
#             ).distribute(protocols=[make_bound_states])
#             client.close()
#         cluster.scale(0)
#         cluster.close()
#     print("distributed run complete")

### TODO Load resulting scorefiles of bound and free states

In [5]:
# sys.path.insert(0, os.getcwd())
# from crispy_shifty.utils.io import parse_scorefile_linear

# bound_scores_df = parse_scorefile_linear(
#     os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/scores.json")
# )

# free_scores_df = parse_scorefile_linear(
#     os.path.join(os.getcwd(), "scaffolds/02_make_free_states/scores.json")
# )

### TODO Save a list of outputs

In [6]:
# with open(
#     os.path.join(os.getcwd(), "scaffolds/02_make_bound_states/bound_states.list"), "w"
# ) as f:
#     for path in tqdm(bound_scores_df.index):
#         print(path, file=f)
# with open(
#     os.path.join(os.getcwd(), "scaffolds/02_make_free_states/free_states.list"), "w"
# ) as f:
#     for path in tqdm(free_scores_df.index):
#         print(path, file=f)

### Unused blocks
`"-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",` isn't needed.  
`-precompute_ig true \-precompute_ig true \`
`-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5 \`

In [7]:
%%time 
import pyrosetta

pyrosetta.init(
    "\
    -corrections::beta_nov16 \
    -indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5 "
)


sys.path.insert(0, "~/projects/crispy_shifty/") # TODO projects
from crispy_shifty.protocols.design import one_state_design_bound_state

t = one_state_design_bound_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/scaffolds/02_make_bound_states/decoys/0000/notebooks_02_make_bound_states_6f2b82a6c1404a7389d1fe5797fb9176.pdb.bz2',
        }
)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.31+release.c7009b3115c22daa9efe2805d9d1ebba08426a54 2021-08-07T10:04:12] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r292 2021.31+release.c7009b3 c7009b3115c22daa9efe2805d9d1ebba08426a54 http://www.pyrosetta.org 2021-08-07T10:04:12
core.init: {0} command: PyRosetta -corrections::beta_nov16 -indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5 -database /home/pleung/.conda/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=369645880 seed_offset=0 real_seed=369645880 threa

In [8]:
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

0.00 min: Generated interface selector
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=false boundary=true surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.sele

AttributeError: 'pyrosetta.rosetta.protocols.simple_filters.Contact' object has no attribute 'selector1_'

In [11]:
cms = pyrosetta.rosetta.protocols.simple_filters.ContactMolecularSurfaceFilter.setup_from_selectors()

AttributeError: type object 'pyrosetta.rosetta.protocols.simple_filters.Contact' has no attribute 'setup_from_selectors'